<a href="https://colab.research.google.com/github/Zia-Ul-Hasan/Churn-Prediction-IBM-telco/blob/main/scratchcode6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd
import numpy as np
import joblib
import random
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
path='/content/Telco_customer_churn.xlsx'
df = pd.read_excel(path)
print (df.info())
#removing unimportant columns from the dataset
df= df.drop(['CustomerID','Country','Count','State','Zip Code','Lat Long','Longitude','Latitude','City', 'Churn Score','Churn Label'], axis=1)
print (df.info())
df.to_csv('required_data_only.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

In [88]:
def unique_values_col (df):
 for column in df:
  if df[column].dtypes=='object':
   print(f'{column} : {df[column].unique()}')
 return None


def replace_t(df):
    print(unique_values_col(df))
    for column in df.columns:
        df.loc[df[column] == 'No internet service', column] = 'No'
        df.loc[df[column] == 'No phone service', column] = 'No'
    print(unique_values_col(df))
    return

replace_t(df)

df['Monthly Charges']= df['Monthly Charges'].astype(float)
df['CLTV']= df['CLTV'].astype(float)
df['Total Charges']= pd.to_numeric(df['Total Charges'], errors='coerce')
df['Tenure Months']= pd.to_numeric(df['Tenure Months'], errors='coerce')

Gender : ['Male' 'Female']
Senior Citizen : ['No' 'Yes']
Partner : ['No' 'Yes']
Dependents : ['No' 'Yes']
Phone Service : ['Yes' 'No']
Multiple Lines : ['No' 'Yes' 'No phone service']
Internet Service : ['DSL' 'Fiber optic' 'No']
Online Security : ['Yes' 'No' 'No internet service']
Online Backup : ['Yes' 'No' 'No internet service']
Device Protection : ['No' 'Yes' 'No internet service']
Tech Support : ['No' 'Yes' 'No internet service']
Streaming TV : ['No' 'Yes' 'No internet service']
Streaming Movies : ['No' 'Yes' 'No internet service']
Contract : ['Month-to-month' 'Two year' 'One year']
Paperless Billing : ['Yes' 'No']
Payment Method : ['Mailed check' 'Electronic check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Total Charges : [108.15 151.65 820.5 ... 7362.9 346.45 6844.5]
Churn Reason : ['Competitor made better offer' 'Moved' 'Competitor had better devices'
 'Competitor offered higher download speeds'
 'Competitor offered more data' 'Price too high' 'Product dissatisfac

In [89]:
y1= df['Churn Value']
y2= df['Churn Reason']
drop_c= ['Churn Value','Churn Reason']
x = df.drop(columns=drop_c)

#for finding columns that require label encoding, one hot encoding and numerical columns
num_col=[]
LE_col=[]
OE_col=[]
for col in x:
  if pd.api.types.is_numeric_dtype(x[col]):
    num_col.append(col)
  else:
   if x[col].nunique() == 2:
    LE_col.append(col)
   elif x[col].nunique()>2:
    OE_col.append(col)
print (f' the column to be label encoded:{LE_col}')
print (f' the column to be One Hot Endoded encoded:{OE_col}')
print(f'the numerical columns are: {num_col}')

 the column to be label encoded:['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Paperless Billing']
 the column to be One Hot Endoded encoded:['Internet Service', 'Contract', 'Payment Method']
the numerical columns are: ['Tenure Months', 'Monthly Charges', 'Total Charges', 'CLTV']


In [90]:
y1= df['Churn Value']
y2= df['Churn Reason']
drop_c= ['Churn Value','Churn Reason']
x = df.drop(columns=drop_c)

#for finding columns that require label encoding, one hot encoding and numerical columns
num_col=[]
LE_col=[]
OE_col=[]
for col in x:
  if pd.api.types.is_numeric_dtype(x[col]):
    num_col.append(col)
  else:
   if x[col].nunique() == 2:
    LE_col.append(col)
   elif x[col].nunique()>2:
    OE_col.append(col)
print (f' the column to be label encoded:{LE_col}')
print (f' the column to be One Hot Endoded encoded:{OE_col}')
print(f'the numerical columns are: {num_col}')
#splitting the dataset

x_train, x_test, y1_train, y1_test = train_test_split(x,y1, test_size=0.2, random_state=42)
x_train, x_test, y2_train, y2_test = train_test_split(x,y2, test_size=0.2, random_state=42)

#dummy/ one hot encoding
OE = OneHotEncoder(sparse_output=False)
def dummy_encoding (tt):
 OEd = OE.fit_transform(tt[OE_col])
 OEd_df = pd.DataFrame(OEd, columns=OE.get_feature_names_out(OE_col))
 tt= tt.drop(columns=OE_col)
 tt= pd.concat([tt,OEd_df], axis=1)
 return tt
x_train= dummy_encoding(x_train)
x_test= dummy_encoding(x_test)

#Label Encoding
LE = LabelEncoder()
def Label_encoding(tt):
 for col in LE_col:
  tt[col]=LE.fit_transform(tt[col])
 return tt

y2_train.to_csv("preprocssed_y2_train.csv")
x_train=Label_encoding(x_train)
x_test=Label_encoding(x_test)

y2_train=LE.fit_transform(y2_train)
y2_test=LE.fit_transform(y2_test)
y2_train = pd.DataFrame(y2_train, columns=['Churn Reason'])
y2_test = pd.DataFrame(y2_test, columns=['Churn Reason'])

print(x_train)
print(x_test)
print(y1_test.unique())
print(y1_train.unique())
print(y2_train['Churn Reason'].unique())
print(y2_train['Churn Reason'].unique())

x_train.to_csv("preprocssed_x_train.csv")
x_test.to_csv("preprocssed_x_test.csv")

y1_train.to_csv("preprocssed_y1_train.csv")
y1_test.to_csv("preprocssed_y1_test.csv")
y2_train.to_csv("preprocssed_y2_train.csv")
y2_test.to_csv("preprocssed_y2_test.csv")

 the column to be label encoded:['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Paperless Billing']
 the column to be One Hot Endoded encoded:['Internet Service', 'Contract', 'Payment Method']
the numerical columns are: ['Tenure Months', 'Monthly Charges', 'Total Charges', 'CLTV']
      Gender  Senior Citizen  Partner  Dependents  Tenure Months  \
2142       1               0        1           0           71.0   
1623       0               0        0           0           16.0   
6074       0               0        0           0           32.0   
1362       0               1        1           0           24.0   
6754       0               0        1           1           20.0   
...      ...             ...      ...         ...            ...   
5597       2               2        2           2            NaN   
5603       2           